<a href="https://colab.research.google.com/github/ayaz-ncr/100ml/blob/master/00_any_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
file1 = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [9]:
list_shape = [train.shape, test.shape]
list_shape

[(19579, 3), (8392, 2)]

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train.author.values)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train.text.values, y, test_size = 0.2)

In [23]:
print(X_train.shape)
print(X_valid.shape)

(15663,)
(3916,)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=3, ngram_range=(1,3))
tfidf.fit(list(X_train)+list(X_valid))
X_train_tfidf = tfidf.transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)

In [0]:
import numpy as np
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [12]:
from sklearn.linear_model import LogisticRegression
cls = LogisticRegression()
cls.fit(X_train_tfidf, y_train)
predictions = cls.predict_proba(X_valid_tfidf)
print("logloss: %0.3f" % multiclass_logloss(y_valid, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 0.628


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(list(X_train)+list(X_valid))
X_train_cv = cv.transform(X_train)
X_valid_cv = cv.transform(X_valid)

In [14]:
from sklearn.linear_model import LogisticRegression
cls2 = LogisticRegression()
cls2.fit(X_train_cv, y_train)
predictions = cls2.predict_proba(X_valid_cv)
print("logloss: %0.3f" % multiclass_logloss(y_valid, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 0.456


In [15]:
cls2.predict_proba(X_valid_cv)

array([[2.94406880e-02, 2.66229077e-02, 9.43936404e-01],
       [5.50173409e-04, 9.83461415e-01, 1.59884113e-02],
       [1.70351711e-01, 1.51002313e-02, 8.14548057e-01],
       ...,
       [2.05093908e-01, 6.36757596e-01, 1.58148496e-01],
       [2.43682143e-01, 5.36044349e-02, 7.02713422e-01],
       [1.31537817e-02, 1.49652527e-03, 9.85349693e-01]])

In [16]:
# Fitting a simple Naive Bayes on TFIDF
from sklearn.naive_bayes import MultinomialNB
cls_nb = MultinomialNB()
cls_nb.fit(X_train_tfidf, y_train)
predictions = cls_nb.predict_proba(X_valid_tfidf)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.587 


In [17]:
# Fitting a simple Naive Bayes on Counts
cls_nb = MultinomialNB()
cls_nb.fit(X_train_cv, y_train)
predictions = cls_nb.predict_proba(X_valid_cv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.475 


In [0]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=120)
svd.fit(X_train_tfidf)
X_train_svd = svd.transform(X_train_tfidf)
X_valid_svd = svd.transform(X_valid_tfidf)

from sklearn.preprocessing import StandardScaler
# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = StandardScaler()
scl.fit(X_train_svd)
X_train_svd_scl = scl.transform(X_train_svd)
X_valid_svd_scl = scl.transform(X_valid_svd)

In [20]:
# Fitting a simple SVM
from sklearn.svm import SVC
cls_svm = SVC(C=1.0, probability=True) # since we need probabilities
cls_svm.fit(X_train_svd_scl, y_train)
predictions = cls_svm.predict_proba(X_valid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.691 


In [22]:
import xgboost as xgb
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(X_train_tfidf.tocsc(), y_train)
predictions = clf.predict_proba(X_valid_tfidf.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.683 


In [23]:
# Fitting a simple xgboost on cv
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(X_train_cv.tocsc(), y_train)
predictions = clf.predict_proba(X_valid_cv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.682 


In [24]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(X_train_svd, y_train)
predictions = clf.predict_proba(X_valid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.744 


In [0]:
from sklearn.metrics import make_scorer
mll_scorer = make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [0]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

from sklearn.pipeline import Pipeline
# Create the pipeline 
clf = Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [0]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [37]:
from sklearn.model_selection import GridSearchCV
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(X_train_tfidf, y_train)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.8s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 

Best score: -0.713
Best parameters set:
	lr__C: 10
	lr__penalty: 'l1'
	svd__n_components: 180


In [0]:
# GridSearch 2

In [0]:
from google.colab import files
file_glove = files.upload()

In [0]:
# load the GloVe vectors in a dictionary:
from tqdm import tqdm
embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))